In [3]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

bets = DeltaTable("/Users/andreikirpichev/parquet/bets/")
bets = bets.to_pandas()
#bets['label'] = np.where(bets['stake_eur'] - bets['weekday_stake'] > 0, 1, 0)
bets['label'] = 1

In [4]:
test_betmarkets = DeltaTable("/Users/andreikirpichev/parquet/test_betmarkets/")
test_betmarkets = test_betmarkets.to_pandas()
test_betmarkets.head(1)

,contest_id,section,point,market,away,start,end,home,league,region,sport,placement_odds
0,FB-216767,RT,goal,BothTeamsToScore,UE Engordany,2022-10-22 14:53:19.559,2022-10-23 16:30:00,UE Santa Coloma,Primera Division,AD,Football,1.72


In [5]:
bets_populated = pd.read_parquet('/Users/andreikirpichev/parquet/bets_populated_weighted.gzip')

In [6]:
from sklearn.preprocessing import LabelEncoder
betmarket_categorical_features = ['section', 'point', 'market', 'region', 'contest_id', 'league', 'home', 'away']
categorical_features = betmarket_categorical_features + ['country']

le = LabelEncoder()

# Convert the variables to numerical
def encode(arr, features):
    for i in range(len(features)):
        encoded = le.fit_transform(arr[features[i]])
        arr[features[i]] = encoded

encode(test_betmarkets, betmarket_categorical_features)
encode(bets_populated, categorical_features)

In [7]:
# Create features and target
features = categorical_features + ['age', 'placement_odds']
features_meta = features + ['placement_time', 'customer_id', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']
X = bets_populated[features_meta]
y = bets_populated[['label']]

# show rows with NaN values
#X[X.isna().any(axis=1)]

# Make necessary imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101, shuffle=False)

print('train: from', X_train['placement_time'].min(), ' to ', X_train['placement_time'].max(), X_train.shape)
print('test: from', X_test['placement_time'].min(), ' to ', X_test['placement_time'].max(), X_test.shape)
threshhold = X_train['placement_time'].max()

X_train = X_train[features]

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train.values.ravel())

train: from 2022-09-01 00:00:07.176000  to  2022-10-17 20:05:48.066000 (6297092, 17)
test: from 2022-10-17 20:05:48.066000  to  2022-10-28 14:23:42.051000 (1574274, 17)


/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [8]:
X_test.dtypes

section                    int64
point                      int64
market                     int64
region                     int64
contest_id                 int64
league                     int64
home                       int64
away                       int64
country                    int64
age                        int64
placement_odds           float64
placement_time    datetime64[ns]
customer_id               object
weekday_stake            float64
section_2w               float64
point_2w                 float64
market_2w                float64
dtype: object

In [28]:
import warnings
warnings.filterwarnings('ignore')

# Make prediction
predictions = model.predict_proba(X_test[features])

result = X_test[:]
result['label'] = y_test
result['prediction'] = predictions.tolist()
result.head()

,section,point,market,region,contest_id,league,home,away,country,age,placement_odds,placement_time,customer_id,weekday_stake,section_2w,point_2w,market_2w,label,prediction
136724,7,2,4,104,26629,243,3831,6200,3,89,2.4756,2022-10-17 20:05:48.066,17297810,189.37,53.72,72.25,48.81,0,"[0.26436097327179775, 0.7356390267282022]"
3419903,5,2,21,27,5555,186,7223,28,5,53,5.1000,2022-10-17 20:05:48.066,15834910,140.24,15.67,224.73,83.56,0,"[0.7535917210829757, 0.24640827891702435]"
3139673,1,1,4,4,30380,68,7270,4215,2,33,1.9000,2022-10-17 20:05:49.417,11339210,0.20,0.17,0.14,0.23,1,"[0.6280268382729739, 0.3719731617270261]"
3139672,1,1,4,27,35353,161,2346,7221,2,33,4.0000,2022-10-17 20:05:49.417,11339210,0.20,0.17,0.14,0.23,1,"[0.8142854027247666, 0.18571459727523337]"
3139675,2,1,4,40,29343,156,1778,5609,2,33,2.6000,2022-10-17 20:05:49.417,11339210,0.18,0.16,0.14,0.23,1,"[0.6599602159927315, 0.34003978400726853]"


In [10]:
# Import library for metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Print metrics
print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions))
print("Recall:", recall_score(y_test, predictions))

Accuracy: 0.7388237371639245
Precision: 0.7869707413610285
Recall: 0.6629209107400391


In [11]:
X_test.shape

(1574274, 17)

In [17]:
X_test.head(3)

,section,point,market,region,contest_id,league,home,away,country,age,placement_odds,placement_time,customer_id,weekday_stake,section_2w,point_2w,market_2w
136724,7,2,4,104,26629,243,3831,6200,3,89,2.4756,2022-10-17 20:05:48.066,17297810,189.37,53.72,72.25,48.81
3419903,5,2,21,27,5555,186,7223,28,5,53,5.1000,2022-10-17 20:05:48.066,15834910,140.24,15.67,224.73,83.56
3139673,1,1,4,4,30380,68,7270,4215,2,33,1.9000,2022-10-17 20:05:49.417,11339210,0.20,0.17,0.14,0.23


In [50]:
pd.options.mode.chained_assignment = None
size = 1000
shuffled = X_test[:].sample(frac=1)

progress = 0
pred_sum = 0
base_sum = 0
for index, row in shuffled.head(size).iterrows():
    current_betmarkets = test_betmarkets[(test_betmarkets['start'] <= row['placement_time']) &
                                         (test_betmarkets['end'] >= row['placement_time'])]
    customer_bet_features = ['country', 'age']#, 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']
    current_betmarkets[customer_bet_features] = row[customer_bet_features]
    bets_2w = bets_populated[(bets_populated['placement_time'] > row['placement_time'] - np.timedelta64(14,'D')) &
                   (bets_populated['placement_time'] < row['placement_time'])]
    def set_avg_2w(bm_type):
        type_2w = bets_2w.groupby(['customer_id',bm_type],as_index=False)['stake_eur'].mean('')
        type_by_customer = type_2w[type_2w['customer_id'] == row['customer_id']][[bm_type, 'stake_eur']]
        type_by_customer[bm_type]=type_by_customer[bm_type].astype(int)
        current_betmarkets[bm_type + '_2w'] = pd.merge(current_betmarkets, type_by_customer, on=bm_type)[['stake_eur']]
        current_betmarkets[bm_type + '_2w'] = current_betmarkets[bm_type + '_2w'].fillna(0)

    set_avg_2w('section')
    set_avg_2w('point')
    set_avg_2w('market')

    predictions = model.predict_proba(current_betmarkets[features])
    current_betmarkets['prediction'] = [tuple[1] for tuple in predictions]
    recommended = current_betmarkets.query('prediction == prediction.max()')
    model_predicted = recommended.iloc[0]

    customer_bets = bets_2w[bets_2w['customer_id'] == row['customer_id']]
    if customer_bets.empty:
        baseline_predicted['market'] = 4
        baseline_predicted['point'] = 7
        baseline_predicted['section'] = 37
    else:
        last_customer_bet = customer_bets.query('placement_time == placement_time.max()')
        baseline_predicted = last_customer_bet.iloc[0]

    customerContestIdInteractions = bets_2w[(bets_2w['customer_id'] == row['customer_id']) &
                                            (bets_2w['contest_id'] == row['contest_id'])]
    if customerContestIdInteractions.empty:
        mostPopular = bets_2w[['contest_id', 'region', 'league']].mode().iloc[0]
        baseline_predicted[['region', 'league']] = mostPopular[['region', 'league']]
    else:
        baseline_predicted[['region', 'league']] = customerContestIdInteractions.iloc[0]


    def hitRate(predicted, sum):
        res = ((1 if predicted['league'] == row['league'] else 0) +
               (1 if predicted['region'] == row['region'] else 0) +
               (1 if predicted['market'] == row['market'] else 0) +
               (1 if predicted['point'] == row['point'] else 0) +
               (1 if predicted['section'] == row['section'] else 0)) / 5
        sum += res
        return sum

    progress += 1
    if progress % 50 == 0:
        print("Mode Hit Rate: ", round(hitRate(model_predicted, pred_sum) / progress * 100, 2), "%",
              "baseline: ", round(hitRate(baseline_predicted, base_sum) / progress * 100, 2), "%",
              " processed ", round(progress / size * 100, 2), "%")

Mode Hit Rate:  0.4 % baseline:  1.6 %  processed  5.0 %
Mode Hit Rate:  0.2 % baseline:  0.0 %  processed  10.0 %
Mode Hit Rate:  0.13 % baseline:  0.27 %  processed  15.0 %
Mode Hit Rate:  0.1 % baseline:  0.2 %  processed  20.0 %
Mode Hit Rate:  0.08 % baseline:  0.4 %  processed  25.0 %
Mode Hit Rate:  0.07 % baseline:  0.07 %  processed  30.0 %
Mode Hit Rate:  0.06 % baseline:  0.17 %  processed  35.0 %
Mode Hit Rate:  0.0 % baseline:  0.05 %  processed  40.0 %
Mode Hit Rate:  0.04 % baseline:  0.22 %  processed  45.0 %
Mode Hit Rate:  0.04 % baseline:  0.2 %  processed  50.0 %
Mode Hit Rate:  0.0 % baseline:  0.0 %  processed  55.0 %
Mode Hit Rate:  0.03 % baseline:  0.0 %  processed  60.0 %
Mode Hit Rate:  0.03 % baseline:  0.03 %  processed  65.0 %
Mode Hit Rate:  0.0 % baseline:  0.0 %  processed  70.0 %
Mode Hit Rate:  0.0 % baseline:  0.0 %  processed  75.0 %
Mode Hit Rate:  0.05 % baseline:  0.03 %  processed  80.0 %
Mode Hit Rate:  0.02 % baseline:  0.12 %  processed  85.0

In [ ]:
import random
cols = bets.columns.tolist()
cols.remove("placement_time")
populated = pd.DataFrame()
for i in range(len(cols)):
    col = cols[i]
    value_counts = bets[col].value_counts()
    population = value_counts.index.tolist()
    weights = value_counts.values.tolist()
    populated[col] = random.choices(population, weights=None, k=bets.shape[0])

value_counts = bets["placement_time"].value_counts()
population = value_counts.index.tolist()
weights = value_counts.values.tolist()
datetimes = pd.DatetimeIndex(population, dtype='datetime64[ns]', freq=None)
populated["placement_time"] = random.choices(datetimes.values, k=bets.shape[0])

populated['label'] = 0
bets_populated = pd.concat([bets, populated])
bets_populated = bets_populated.sort_values('placement_time')
bets_populated.to_parquet('/Users/andreikirpichev/parquet/bets_populated_weighted.gzip',
            compression='gzip')